In [2]:
%pip install xgboost
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np

In [4]:
from Data_preparation import create_fish_pipeline, prepare_fish_data


In [9]:
def evaluate_xgb(X_train, y_train, X_dev, y_dev):
    print("Evaluating XGBoost Regressor...")

    # Define the hyperparameter grid search to try combinations of these hyperparameters.
    param_grid = {
        'algo__n_estimators': [50, 100],
        'algo__max_depth': [3, 5],
        'algo__learning_rate': [0.05, 0.1],
        'algo__subsample': [0.8, 1.0]
    }

    # This here uses the pipeline to handle missing values, scaling, encoding, etc for teh dataset.
    pipeline = create_fish_pipeline()

    # This combines the preprocessing and XGBoost model into one clean pipeline.
    pipeline_with_algo = Pipeline(steps=[
        ('preprocessor', pipeline),
        ('algo', XGBRegressor(
            objective='reg:squarederror',
            random_state=42
        ))
    ])

    grid_search = GridSearchCV(
        pipeline_with_algo, param_grid,
        cv=3,  # 3-fold cross-validation
        scoring='r2',  # Use R² as the evaluation metric
        verbose=1  # Show progress in terminal
    )
    grid_search.fit(X_train, y_train)

    # This shows us our best model based on cross-validation R² score.
    best_estimator = grid_search.best_estimator_

    # We are making predicitons on the dev set here
    y_pred = best_estimator.predict(X_dev)

    # Here we are calculating the following values
    mse = mean_squared_error(y_dev, y_pred)
    mae = mean_absolute_error(y_dev, y_pred)
    r2 = r2_score(y_dev, y_pred)

    # Shows you the best performance from the training phase and the hyperparameters that gave it.
    print("Grid searching is done!")
    print("Best score (neg MSE):", grid_search.best_score_)
    print("Best hyperparameters:")
    print(grid_search.best_params_)

    return best_estimator, mse, mae, r2


In [10]:
# Step 1: Prepare fish data (split into train/dev/test)
X_train, X_dev, X_test, y_train, y_dev, y_test = prepare_fish_data(ratios=((1/10), (1/10)))

# Step 2: Run hyperparameter tuning on train/dev sets
best_model, dev_mse, dev_mae, dev_r2 = evaluate_xgb(X_train, y_train, X_dev, y_dev)

print("\n----- Dev Set Performance -----")
print("Dev MSE:", dev_mse)
print("Dev MAE:", dev_mae)
print("Dev R²:", dev_r2)

# Step 3: Evaluate best model on test set
y_test_pred = best_model.predict(X_test)

test_mse = mean_squared_error(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("\n----- Test Set Performance -----")
print("Test MSE:", test_mse)
print("Test MAE:", test_mae)
print("Test R²:", test_r2)

Evaluating XGBoost Regressor...
Fitting 3 folds for each of 16 candidates, totalling 48 fits
Grid searching is done!
Best score (neg MSE): 0.7343398167270424
Best hyperparameters:
{'algo__learning_rate': 0.1, 'algo__max_depth': 3, 'algo__n_estimators': 100, 'algo__subsample': 1.0}

----- Dev Set Performance -----
Dev MSE: 0.0006988098230355483
Dev MAE: 0.0061427891355794765
Dev R²: 0.9617233611821583

----- Test Set Performance -----
Test MSE: 0.036535925060766236
Test MAE: 0.009838734049708876
Test R²: 0.8160805460065046


## 🔍 Observations from the Data Stats

### 📉 Missing Values
- Columns like **"AM Transparency"**, **"PM Transparency"**, and **"Spring Temp"** have missing values (their count is less than the total 28,918 rows).
- ✅ Your pipeline handles this using `SimpleImputer` and `KNNImputer`.

### 📊 Wide Value Ranges
- **Transparency** values go above **1200**, so scaling is needed — ✅ you're using `StandardScaler`.
- **Fish counts** range from **220 to 51,827**, creating large variance — yet your model handled it well (**Test R² ~0.81**).

### 🎯 Survival Rate
- "Fish survival rate" ranges from **~68% to 100%**
- Mean ≈ **99.97%** → Highly **imbalanced**, as most fish survive.
- ✅ Your model's low **MAE (≈ 0.0098)** shows it's accurately predicting survival rates even in this tight range.

### 📝 Other Notes
- **Rain values** include 0 — makes sense seasonally.
- **Temperature and rainfall** columns are continuous — great for regression models.

---

## ✅ Why Your Pipeline Is Working

Your preprocessing pipeline and model work well because:

| Problem                              | Solution in Pipeline                   |
|--------------------------------------|----------------------------------------|
| Missing transparency/temp values     | `KNNImputer`, `SimpleImputer`          |
| Large numeric value ranges           | `StandardScaler`                       |
| Categorical feeding/location columns | `OneHotEncoder`                        |
| Special handling for `"Morts"`       | Filled with `0` using `SimpleImputer` ✅ |

---

## ✅ Why the Model Performed Well

| Metric     | Result     | Interpretation                              |
|------------|------------|----------------------------------------------|
| Dev R²     | 0.96       | Extremely good fit to known (dev) data       |
| Test R²    | 0.81       | Strong generalization to unseen (test) data  |
| MAE        | ~0.0098    | Average error is <1% of survival rate — ✅ very precise |


In [26]:
# Get the preprocessor from the trained pipeline
preprocessor = best_model.named_steps['preprocessor']

# Transform the training data
X_train_transformed = preprocessor.transform(X_train)

# If it's a sparse matrix, convert it to dense
if hasattr(X_train_transformed, "toarray"):
    X_train_transformed = X_train_transformed.toarray()

# Convert to DataFrame
X_train_df = pd.DataFrame(X_train_transformed)

# Optional: Add target column
X_train_df["Survival Rate"] = y_train.reset_index(drop=True)

# Preview
print(X_train_df)

              0         1         2         3         4         5         6  \
0     -0.050199 -0.078393 -1.151880 -1.067648 -0.311425 -0.317489 -0.340997   
1      0.044104  1.402440  0.841232  0.551391 -0.342785 -0.317489 -0.352084   
2     -0.050199 -0.882582 -0.198653 -0.115272 -0.342785 -0.348926 -0.521603   
3     -0.003048 -0.378859  0.104647 -0.258129 -0.342785 -0.348926 -1.486657   
4     -0.050199  1.140310  1.144531  1.456148  0.942978  0.782803 -0.345348   
...         ...       ...       ...       ...       ...       ...       ...   
23131 -0.050199  1.035716  1.144531  0.503772 -0.342785 -0.348926 -0.192387   
23132 -0.050199  0.044681  1.231188  1.122816 -0.342785 -0.348926 -0.152253   
23133  0.044104 -1.347008 -1.975122 -1.877168  0.221696  0.185502 -0.275744   
23134 -0.050199 -0.937113 -1.325194 -1.591455  0.221696  0.279812 -0.432914   
23135 -0.050199  0.035668  0.537932 -0.067653 -0.311425 -0.254615 -0.295390   

              7          8          9  ...  100  10

In [27]:
X_train_df.to_excel("../Data/Prepared/preprocessed_train_data.xlsx", index=False)
